# Cell 1: 导入 (Imports)

In [15]:
import logging
import re
import sys
from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

# (Review #1) 导入 PIL 并设置以处理截断图像
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# (Review #3) 导入 pyarrow 用于更稳健的 Parquet 写入
import pyarrow as pa
import pyarrow.parquet as pq

# 确保 open_clip 在 Python 路径中
# sys.path.append('/path/to/your/open_clip/src')
import open_clip

# 配置日志记录
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

# Cell 2: 配置 (Configuration)

In [42]:
# --- 核心配置 ---
# 🛡️ 安全第一: 默认在预演模式下运行
DRYRUN = False

# (Review #12) 设置随机种子以保证可复现性
def set_seed(seed=2025):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(2025)

# 路径配置
BASE_DIR = Path("/cwStorage/nodecw_group/jijh/trained_models/omiclip_base_model/")
TRAIN_MANIFEST = BASE_DIR / "train_manifest.csv"
VALID_MANIFEST = BASE_DIR / "validation_manifest.csv"
MODEL_CHECKPOINT = BASE_DIR / "omiclip_epoch_50.pt"

# (Review #8) 检查关键文件是否存在
for p in [TRAIN_MANIFEST, VALID_MANIFEST, MODEL_CHECKPOINT]:
    assert p.exists(), f"关键文件缺失: {p}"

# (Review #4) 定义更鲁棒的正则表达式
FILENAME_REGEX = re.compile(r'(.+?)_(-?\d+)_(-?\d+)\.(png|jpg|jpeg|tif|tiff)$', re.IGNORECASE)

# 模型配置
MODEL_NAME = "ViT-B-32"

# 邻域计算配置
K_NEIGHBORS = 6

# 批处理和设备配置
BATCH_SIZE = 128  # (Review #9) 降低默认批次大小以提高兼容性

# (Review #2) 稳健的设备选择逻辑
def pick_device(prefer_cuda_index: int = 2) -> str:
    if torch.cuda.is_available():
        n = torch.cuda.device_count()
        idx = prefer_cuda_index if prefer_cuda_index < n else 0
        return f"cuda:{idx}"
    elif torch.backends.mps.is_available():
        return "mps"
    return "cpu"
DEVICE = pick_device(prefer_cuda_index=2)

# 输出配置
ARTIFACTS_DIR = Path("/cwStorage/nodecw_group/jijh/yuanspace_data/artifacts")
OUTPUT_NODES_PATH = ARTIFACTS_DIR / "nodes.parquet"
OUTPUT_EDGES_PATH = ARTIFACTS_DIR / "edges.parquet"
OUTPUT_IMG_EMBED_PATH = ARTIFACTS_DIR / "image_embeds.npy"
OUTPUT_TXT_EMBED_PATH = ARTIFACTS_DIR / "text_embeds.npy"

# (Review #13) 显式创建目录
if not DRYRUN:
    ARTIFACTS_DIR.mkdir(exist_ok=True)
    (ARTIFACTS_DIR / "clip_cache").mkdir(exist_ok=True, parents=True)
    
logging.info(f"设备: {DEVICE}")
logging.info(f"预演模式 (DRYRUN): {DRYRUN}")

2025-09-03 11:30:13,981 - INFO - 设备: cuda:2
2025-09-03 11:30:13,981 - INFO - 预演模式 (DRYRUN): False


# Cell 3: 核心逻辑 - 数据加载与处理 (Core Logic - Data Loading & Processing)

In [37]:
import os
from concurrent.futures import ProcessPoolExecutor
from tqdm.contrib.concurrent import process_map

# (Review #1) 导入 PIL 并设置以处理截断图像
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- 新增：健壮的图像验证函数 ---
def is_image_valid(filepath: str) -> bool:
    """
    检查单个图像文件是否有效。
    返回 True 如果文件存在、大小不为零且可以被PIL识别。
    """
    try:
        p = Path(filepath)
        # 1. 快速检查：是否存在和大小是否为0
        if not p.exists() or p.stat().st_size == 0:
            return False
        # 2. 深度检查：PIL能否打开并验证
        with Image.open(p) as img:
            img.verify()  # 快速检查文件头和元数据
        return True
    except Exception:
        # 捕获所有异常 (FileNotFoundError, PIL.UnidentifiedImageError, etc.)
        return False
# ------------------------------------

def parse_path(path_str: str) -> Tuple[str, int, int]:
    """从图像路径中解析 sample_id, x, y 坐标。"""
    m = FILENAME_REGEX.search(Path(path_str).name)
    if not m:
        raise ValueError(f"无法解析路径格式: {path_str}")
    sample_id, x, y, _ = m.groups()
    return sample_id, int(x), int(y)

def find_malformed_paths(paths: pd.Series) -> List[str]:
    """快速查找所有不符合预期格式的路径。"""
    return [p for p in paths if not FILENAME_REGEX.search(Path(p).name)]

def load_and_prepare_data(manifest_paths: List[Path]) -> pd.DataFrame:
    """加载、合并、验证、解析并索引清单文件。"""
    logging.info(f"从 {manifest_paths} 加载清单...")
    df = pd.concat([pd.read_csv(p) for p in manifest_paths], ignore_index=True)

    required_cols = {"image_path", "gene_sentence"}
    assert required_cols.issubset(df.columns), f"清单缺少列: {required_cols - set(df.columns)}"

    # --- BUG FIX & ROBUSTNESS ---
    # 在加载任何数据前，并行验证所有图像文件的有效性
    logging.info(f"开始并行验证 {len(df)} 个图像文件...")
    image_paths = df['image_path'].tolist()
    # 使用 process_map 并行处理，并显示进度条
    # max_workers=None 会使用所有可用的CPU核心
    is_valid_mask = process_map(is_image_valid, image_paths, max_workers=None, chunksize=100)
    
    num_original = len(df)
    df_valid = df[is_valid_mask].copy() # 使用 .copy() 避免 SettingWithCopyWarning
    num_valid = len(df_valid)
    num_removed = num_original - num_valid
    
    if num_removed > 0:
        logging.warning(f"验证完成。移除了 {num_removed} 个无效或损坏的图像条目。")
    else:
        logging.info("验证完成。所有图像文件均有效。")
    # ----------------------------

    logging.info("预检查所有路径格式...")
    malformed = find_malformed_paths(df_valid['image_path'])
    if malformed:
        logging.error(f"发现 {len(malformed)} 个格式错误的路径。示例: {malformed[:5]}")
    else:
        logging.info("所有路径格式均有效。")

    logging.info("解析文件名以提取坐标和样本ID...")
    path_data = df_valid['image_path'].apply(parse_path)
    
    parsed_info_df = pd.DataFrame(path_data.tolist(), columns=['sample_id', 'x', 'y'], index=df_valid.index)
    df_final = df_valid.join(parsed_info_df)
    df_final = df_final.astype({'sample_id': 'object', 'x': 'int32', 'y': 'int32'})
    
    df_final.reset_index(drop=True, inplace=True) # 使用 drop=True 避免旧索引成为新列
    df_final.reset_index(inplace=True)
    df_final.rename(columns={'index': 'tile_id'}, inplace=True)
    df_final['tile_id'] = df_final['tile_id'].astype('int64')
    
    logging.info(f"数据加载完成。有效图块总数 {len(df_final)}，分布在 {df_final['sample_id'].nunique()} 个样本中。")
    return df_final

def compute_neighborhoods(df: pd.DataFrame, k: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """按样本分组计算k-最近邻，返回节点和边DataFrame。"""
    logging.info(f"开始计算 k={k} 的最近邻...")
    
    edges = []
    
    for sample_id, group in tqdm(df.groupby('sample_id'), desc="处理样本"):
        coords = group[['x', 'y']].values
        n_neighbors = min(k + 1, len(coords))
        if n_neighbors <= 1:
            continue

        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(coords)
        distances, indices = nbrs.kneighbors(coords)
        
        distances, indices = distances[:, 1:], indices[:, 1:]
        
        non_zero_distances = distances[distances > 0]
        sigma = np.percentile(non_zero_distances, 50) if non_zero_distances.size > 0 else 1.0
        sigma = max(sigma, 1e-6)

        weights = np.exp(- (distances ** 2) / (2 * sigma ** 2))
        
        sum_w = weights.sum(axis=1, keepdims=True)
        alpha = np.divide(weights, np.maximum(sum_w, 1e-8))

        src_tile_ids = group['tile_id'].values
        neighbor_tile_ids = src_tile_ids[indices]

        for i in range(len(group)):
            src_id = src_tile_ids[i]
            for j in range(indices.shape[1]):
                edges.append((
                    src_id,
                    neighbor_tile_ids[i, j],
                    distances[i, j],
                    weights[i, j],
                    alpha[i, j]
                ))

    logging.info("邻域计算完成。")
    edges_df = pd.DataFrame(edges, columns=[
        "src_tile_id", "nbr_tile_id", "distance", "weight", "alpha"
    ]).astype({
        "src_tile_id": "int64", "nbr_tile_id": "int64",
        "distance": "float32", "weight": "float32", "alpha": "float32"
    })

    nodes_df = df[['tile_id', 'sample_id', 'x', 'y', 'image_path', 'gene_sentence']].copy()
    return nodes_df, edges_df

# Cell 4: 核心逻辑 - 嵌入缓存 (Core Logic - Embedding Caching)

In [38]:
class ClipDataset(Dataset):
    """用于批量提取特征的 PyTorch Dataset。"""
    def __init__(self, df: pd.DataFrame, preprocess_fn, tokenizer):
        self.df = df
        self.preprocess = preprocess_fn
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = self.preprocess(Image.open(row['image_path']).convert("RGB"))
        # (Review #5) 确保始终传递列表给 tokenizer
        text = self.tokenizer([row['gene_sentence']])[0]
        return image, text

def cache_embeddings(df: pd.DataFrame, model_name: str, checkpoint_path: Path, device: str, batch_size: int) -> Tuple[np.ndarray, np.ndarray]:
    """为所有图块生成图像和文本嵌入。"""
    logging.info(f"加载模型 {model_name} 从检查点 {checkpoint_path}...")
    
    model, _, preprocess_val = open_clip.create_model_and_transforms(
        model_name, pretrained=None, cache_dir=str(ARTIFACTS_DIR / "clip_cache")
    )
    
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    state_dict = checkpoint.get('state_dict', checkpoint)
    if all(key.startswith('module.') for key in state_dict.keys()):
        state_dict = {k[7:]: v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    
    model.to(device)
    model.eval()
    
    tokenizer = open_clip.get_tokenizer(model_name)
    dataset = ClipDataset(df, preprocess_val, tokenizer)
    
    # (Review #9) 优化 DataLoader
    pin_memory = device.startswith("cuda")
    num_workers = 16
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=pin_memory, persistent_workers=(num_workers > 0)
    )
    
    image_embeds, text_embeds = [], []
    # (Review #9) 使用 torch.inference_mode()
    with torch.inference_mode():
        for images, texts in tqdm(dataloader, desc="提取嵌入"):
            images = images.to(device, non_blocking=True)
            texts = texts.to(device, non_blocking=True)
            
            image_features = model.encode_image(images)
            text_features = model.encode_text(texts)
            
            image_features = F.normalize(image_features, p=2, dim=-1)
            text_features = F.normalize(text_features, p=2, dim=-1)
            
            image_embeds.append(image_features.cpu().numpy())
            text_embeds.append(text_features.cpu().numpy())

    img_emb_array = np.vstack(image_embeds).astype("float32")
    txt_emb_array = np.vstack(text_embeds).astype("float32")
    
    logging.info("嵌入提取完成。")
    return img_emb_array, txt_emb_array

# Cell 5: 主流程与交付 (Main Orchestration & Delivery)

In [40]:
def run_step0_preprocessing():
    """执行Step 0的完整流程。"""
    # 1. 加载和准备数据
    base_df = load_and_prepare_data([TRAIN_MANIFEST, VALID_MANIFEST])
    
    # 2. 计算邻域，得到节点和边表
    nodes_df, edges_df = compute_neighborhoods(base_df, k=K_NEIGHBORS)
    
    # 3. 缓存嵌入
    img_embeds, txt_embeds = cache_embeddings(
        nodes_df, # 仅需节点信息来提取嵌入
        model_name=MODEL_NAME,
        checkpoint_path=MODEL_CHECKPOINT,
        device=DEVICE,
        batch_size=BATCH_SIZE
    )
    
    # 4. (Review #3, #10) 拆分存储到 Parquet 和 Numpy 文件
    if not DRYRUN:
        logging.info("开始将数据写入磁盘...")
        
        # 保存嵌入
        np.save(OUTPUT_IMG_EMBED_PATH, img_embeds)
        np.save(OUTPUT_TXT_EMBED_PATH, txt_embeds)
        logging.info(f"嵌入已保存到 .npy 文件: {OUTPUT_IMG_EMBED_PATH}, {OUTPUT_TXT_EMBED_PATH}")
        
        # 使用 PyArrow 保存 Parquet 文件，更稳健
        pq.write_table(pa.Table.from_pandas(nodes_df, preserve_index=False), OUTPUT_NODES_PATH)
        logging.info(f"节点元数据已保存到: {OUTPUT_NODES_PATH}")
        
        pq.write_table(pa.Table.from_pandas(edges_df, preserve_index=False), OUTPUT_EDGES_PATH)
        logging.info(f"边(邻域)数据已保存到: {OUTPUT_EDGES_PATH}")
        
        logging.info("所有文件保存成功！")
    else:
        logging.warning("处于预演模式，文件未被写入。")
        logging.info("节点表预览:\n" + str(nodes_df.head()))
        logging.info("边表预览:\n" + str(edges_df.head()))
        logging.info(f"图像嵌入数组形状: {img_embeds.shape}")
        logging.info(f"文本嵌入数组形状: {txt_embeds.shape}")

    return nodes_df, edges_df, img_embeds, txt_embeds

# 执行主流程
nodes_df, edges_df, img_embeds, txt_embeds = run_step0_preprocessing()

2025-09-03 11:04:54,416 - INFO - 从 [PosixPath('/cwStorage/nodecw_group/jijh/trained_models/omiclip_base_model/train_manifest.csv'), PosixPath('/cwStorage/nodecw_group/jijh/trained_models/omiclip_base_model/validation_manifest.csv')] 加载清单...
2025-09-03 11:04:58,560 - INFO - 开始并行验证 952909 个图像文件...


  0%|          | 0/952909 [00:00<?, ?it/s]

2025-09-03 11:10:18,086 - WARNING - 验证完成。移除了 1 个无效或损坏的图像条目。
2025-09-03 11:10:18,088 - INFO - 预检查所有路径格式...
2025-09-03 11:10:22,158 - INFO - 所有路径格式均有效。
2025-09-03 11:10:22,160 - INFO - 解析文件名以提取坐标和样本ID...
2025-09-03 11:10:27,192 - INFO - 数据加载完成。有效图块总数 952908，分布在 475 个样本中。
2025-09-03 11:10:27,318 - INFO - 开始计算 k=6 的最近邻...


处理样本:   0%|          | 0/475 [00:00<?, ?it/s]

2025-09-03 11:10:34,339 - INFO - 邻域计算完成。
2025-09-03 11:10:41,038 - INFO - 加载模型 ViT-B-32 从检查点 /cwStorage/nodecw_group/jijh/trained_models/omiclip_base_model/omiclip_epoch_50.pt...
2025-09-03 11:10:41,041 - INFO - Parsing model identifier. Schema: None, Identifier: ViT-B-32
2025-09-03 11:10:41,042 - INFO - Loaded built-in ViT-B-32 model config.
2025-09-03 11:10:41,042 - INFO - No potential checkpoint path found from config source or pretrained arg.
2025-09-03 11:10:41,043 - INFO - Instantiating model architecture: CLIP
2025-09-03 11:10:42,253 - WARNING - Model ViT-B-32 initialized partially.
2025-09-03 11:10:42,254 - INFO - Final image preprocessing configuration set: {'size': (224, 224), 'mode': 'RGB', 'mean': (0.48145466, 0.4578275, 0.40821073), 'std': (0.26862954, 0.26130258, 0.27577711), 'interpolation': 'bicubic', 'resize_mode': 'shortest', 'fill_color': 0}
2025-09-03 11:10:42,254 - INFO - Model ViT-B-32 creation process complete.
2025-09-03 11:10:43,467 - INFO - Parsing tokenizer i

提取嵌入:   0%|          | 0/7445 [00:03<?, ?it/s]

2025-09-03 11:21:17,215 - INFO - 嵌入提取完成。
2025-09-03 11:21:17,259 - INFO - 开始将数据写入磁盘...
2025-09-03 11:21:22,628 - INFO - 嵌入已保存到 .npy 文件: artifacts/image_embeds.npy, artifacts/text_embeds.npy
2025-09-03 11:21:25,490 - INFO - 节点元数据已保存到: artifacts/nodes.parquet
2025-09-03 11:21:26,659 - INFO - 边(邻域)数据已保存到: artifacts/edges.parquet
2025-09-03 11:21:26,663 - INFO - 所有文件保存成功！


In [43]:
def validate_output_revised():
    """加载并验证新生成的所有文件。"""
    logging.info("开始验证拆分后的文件...")
    
    paths_to_check = [
        OUTPUT_NODES_PATH, OUTPUT_EDGES_PATH, 
        OUTPUT_IMG_EMBED_PATH, OUTPUT_TXT_EMBED_PATH
    ]
    
    for p in paths_to_check:
        if not p.exists():
            logging.error(f"验证失败: 文件不存在！{p}。请确保 DRYRUN=False 并重新运行。")
            return
    logging.info("所有必需的文件都已找到。")

    # 1. 验证节点表 (nodes.parquet)
    nodes = pd.read_parquet(OUTPUT_NODES_PATH)
    assert not nodes.empty, "节点表不应为空"
    assert nodes['tile_id'].is_unique, "节点表中的 tile_id 应唯一"
    assert nodes['tile_id'].dtype == 'int64'
    assert nodes['x'].dtype == 'int32'
    logging.info("✅ 节点表基本结构正确。")

    # 2. 验证边表 (edges.parquet)
    edges = pd.read_parquet(OUTPUT_EDGES_PATH)
    assert not edges.empty, "边表不应为空"
    assert 'src_tile_id' in edges.columns and 'nbr_tile_id' in edges.columns
    assert edges['src_tile_id'].isin(nodes['tile_id']).all(), "边表中的源ID应在节点表中"
    assert edges['nbr_tile_id'].isin(nodes['tile_id']).all(), "边表中的邻居ID应在节点表中"
    assert edges['alpha'].dtype == 'float32'
    max_neighbors = edges.groupby('src_tile_id').size().max()
    assert max_neighbors <= K_NEIGHBORS, f"单个节点的最大邻居数 ({max_neighbors}) 超过 k={K_NEIGHBORS}"
    logging.info("✅ 边表基本结构和引用完整性正确。")

    # 3. 验证嵌入数组 (.npy)
    img_e = np.load(OUTPUT_IMG_EMBED_PATH)
    txt_e = np.load(OUTPUT_TXT_EMBED_PATH)
    assert img_e.shape[0] == len(nodes), "图像嵌入数量应与节点数量匹配"
    assert txt_e.shape[0] == len(nodes), "文本嵌入数量应与节点数量匹配"
    
    # 假设 ViT-B-32 的嵌入维度是 512
    embed_dim = 512
    assert img_e.shape[1] == embed_dim, f"图像嵌入维度错误 (应为 {embed_dim})"
    assert txt_e.shape[1] == embed_dim, f"文本嵌入维度错误 (应为 {embed_dim})"
    logging.info("✅ 嵌入数组形状和维度正确。")

    logging.info("🎉 全部验证成功！数据已准备就绪，可以进入下一步。")

# 运行验证
if not DRYRUN:
    validate_output_revised()
else:
    logging.info("处于预演模式，跳过验证。")

2025-09-03 11:30:35,497 - INFO - 开始验证拆分后的文件...
2025-09-03 11:30:35,500 - INFO - 所有必需的文件都已找到。
2025-09-03 11:30:38,200 - INFO - ✅ 节点表基本结构正确。
2025-09-03 11:30:38,699 - INFO - ✅ 边表基本结构和引用完整性正确。
2025-09-03 11:30:39,842 - INFO - ✅ 嵌入数组形状和维度正确。
2025-09-03 11:30:39,843 - INFO - 🎉 全部验证成功！数据已准备就绪，可以进入下一步。


# Cell 6 Visualization Validation

In [45]:
import random
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_neighborhood(nodes_df: pd.DataFrame, edges_df: pd.DataFrame):
    """
    Randomly select a tile and visualize its position and neighbors in the sample space.
    """
    # Ensure data is loaded
    if 'nodes_df' not in locals() or 'edges_df' not in locals():
        logging.info("Loading data from disk for visualization...")
        nodes_df = pd.read_parquet(OUTPUT_NODES_PATH)
        edges_df = pd.read_parquet(OUTPUT_EDGES_PATH)

    # 1. Random sampling
    # Randomly select a sample ID
    sample_ids_with_neighbors = edges_df['src_tile_id'].map(nodes_df.set_index('tile_id')['sample_id']).unique()
    if len(sample_ids_with_neighbors) == 0:
        logging.warning("No samples found in edge table, cannot perform visualization.")
        return
        
    random_sample_id = random.choice(sample_ids_with_neighbors)
    
    # Get all tiles from this sample
    sample_nodes = nodes_df[nodes_df['sample_id'] == random_sample_id]
    
    # Randomly select an anchor tile from this sample
    anchor_tile = sample_nodes.sample(1).iloc[0]
    anchor_id = anchor_tile['tile_id']
    
    logging.info(f"Visualizing anchor tile ID: {anchor_id} from sample '{random_sample_id}'")

    # 2. Data preparation
    # Get neighbor information for the anchor
    anchor_edges = edges_df[edges_df['src_tile_id'] == anchor_id]
    neighbor_ids = anchor_edges['nbr_tile_id'].tolist()
    
    # Get neighbor coordinates and weights
    neighbor_nodes = nodes_df[nodes_df['tile_id'].isin(neighbor_ids)].merge(
        anchor_edges[['nbr_tile_id', 'alpha']],
        left_on='tile_id',
        right_on='nbr_tile_id'
    )

    # 3. Visualization
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Draw background: all other points in this sample
    ax.scatter(
        sample_nodes['x'], sample_nodes['y'],
        s=5, c='lightgray', alpha=0.5, label='Background (same sample)'
    )
    
    # Draw neighbors
    ax.scatter(
        neighbor_nodes['x'], neighbor_nodes['y'],
        s=50, c='royalblue', alpha=0.8, label=f'Neighbors (k={len(neighbor_nodes)})'
    )
    
    # Draw anchor point
    ax.scatter(
        anchor_tile['x'], anchor_tile['y'],
        s=200, c='red', marker='*', edgecolors='black', label='Anchor'
    )
    
    # Draw connection lines, transparency determined by alpha
    for _, neighbor in neighbor_nodes.iterrows():
        ax.plot(
            [anchor_tile['x'], neighbor['x']],
            [anchor_tile['y'], neighbor['y']],
            color='salmon',
            linewidth=1.5,
            alpha=neighbor['alpha'] * 0.8 + 0.2  # Ensure weakest lines are still visible
        )
        
    # 4. Chart styling
    ax.set_title(f"Neighborhood Visualization (Sample: {random_sample_id}, Anchor ID: {anchor_id})", fontsize=16)
    ax.set_xlabel("X Coordinate", fontsize=12)
    ax.set_ylabel("Y Coordinate", fontsize=12)
    ax.legend(loc='best')
    ax.set_aspect('equal', adjustable='box') # Ensure equal x, y axis ratios
    plt.gca().invert_yaxis() # Image coordinate system Y-axis usually points down
    plt.show()

# --- Run visualization ---
# Assume nodes_df and edges_df are already in memory
if not DRYRUN:
    try:
        visualize_neighborhood(nodes_df, edges_df)
    except NameError:
        # If variables are not in memory, load from files
        nodes_df_viz = pd.read_parquet(OUTPUT_NODES_PATH)
        edges_df_viz = pd.read_parquet(OUTPUT_EDGES_PATH)
        visualize_neighborhood(nodes_df_viz, edges_df_viz)
else:
    logging.info("In dry run mode, skipping visualization.")

# Cell 6.1 Visualization Validation 2

In [47]:
import random
import textwrap
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

def load_image_from_path(path_str: str) -> Image.Image:
    """Safely loads an image from a path."""
    try:
        return Image.open(path_str).convert("RGB")
    except Exception as e:
        logging.error(f"Failed to load image at {path_str}: {e}")
        # Return a placeholder image on failure
        return Image.new('RGB', (224, 224), color = 'red')

def visualize_retrieval(nodes_df: pd.DataFrame = None, edges_df: pd.DataFrame = None):
    """
    Randomly selects an anchor tile, retrieves its raw data (image, text), 
    finds its neighbors, retrieves their raw data, and displays them all.
    """
    # 1. 加载数据 (如果尚未加载)
    if nodes_df is None or edges_df is None:
        logging.info("Visualizer is loading data from disk...")
        if not OUTPUT_NODES_PATH.exists() or not OUTPUT_EDGES_PATH.exists():
            logging.error("Parquet files not found. Please run the main pipeline first.")
            return
        nodes_df = pd.read_parquet(OUTPUT_NODES_PATH)
        edges_df = pd.read_parquet(OUTPUT_EDGES_PATH)

    # 2. 随机抽样一个有意义的锚点
    sample_ids_with_neighbors = edges_df['src_tile_id'].map(nodes_df.set_index('tile_id')['sample_id']).unique()
    if len(sample_ids_with_neighbors) == 0:
        logging.warning("No edges found in the dataset. Cannot visualize a neighborhood.")
        return
        
    random_sample_id = random.choice(sample_ids_with_neighbors)
    
    possible_anchors = nodes_df[
        (nodes_df['sample_id'] == random_sample_id) &
        (nodes_df['tile_id'].isin(edges_df['src_tile_id']))
    ]
    if possible_anchors.empty:
        logging.warning(f"Could not find an anchor with neighbors in sample '{random_sample_id}'. Retrying might help.")
        return
        
    anchor_tile_series = possible_anchors.sample(1).iloc[0]
    anchor_id = anchor_tile_series['tile_id']
    
    # 3. 检索数据
    anchor_image = load_image_from_path(anchor_tile_series['image_path'])
    anchor_text = anchor_tile_series['gene_sentence']
    
    anchor_edges = edges_df[edges_df['src_tile_id'] == anchor_id].sort_values('distance')
    neighbor_ids = anchor_edges['nbr_tile_id'].tolist()
    
    # --- BUG FIX ---
    # 错误原因：之前的 merge 操作遗漏了 'distance' 列。
    # 修复方案：在 merge 的列选择中加入 'distance'。
    neighbor_nodes_df = nodes_df[nodes_df['tile_id'].isin(neighbor_ids)].merge(
        anchor_edges[['nbr_tile_id', 'distance', 'alpha']], # <-- 核心修复点
        left_on='tile_id',
        right_on='nbr_tile_id'
    ).set_index('tile_id').loc[neighbor_ids].reset_index() # 保持排序
    # --- END FIX ---

    neighbor_images = [load_image_from_path(p) for p in neighbor_nodes_df['image_path']]
    neighbor_texts = neighbor_nodes_df['gene_sentence'].tolist()
    
    logging.info(f"Visualizing anchor tile {anchor_id} and its {len(neighbor_ids)} neighbors from sample '{random_sample_id}'.")

    # 4. 可视化
    num_plots = 1 + len(neighbor_ids)
    fig, axes = plt.subplots(1, num_plots, figsize=(num_plots * 4, 4.5))
    
    def wrap_text(text, width=40):
        return textwrap.fill(text, width)

    axes[0].imshow(anchor_image)
    axes[0].set_title(f"Anchor Tile: {anchor_id}\n\n{wrap_text(anchor_text)}", fontsize=10)
    axes[0].axis('off')

    for i, row in enumerate(neighbor_nodes_df.itertuples(index=False)):
        ax_neighbor = axes[i + 1]
        ax_neighbor.imshow(neighbor_images[i])
        title = (
            f"Neighbor {i+1} (ID: {row.tile_id})\n"
            f"Dist: {row.distance:.2f}, Alpha: {row.alpha:.2f}\n\n" # 现在可以安全访问 row.distance
            f"{wrap_text(row.gene_sentence)}"
        )
        ax_neighbor.set_title(title, fontsize=10)
        ax_neighbor.axis('off')
        
    plt.suptitle(f"Data Retrieval for Sample '{random_sample_id}'", fontsize=16, y=1.1)
    plt.tight_layout()
    plt.show()

# --- 运行可视化 ---
if not DRYRUN:
    try:
        # 假设 nodes_df 和 edges_df 可能已在内存中
        visualize_retrieval(nodes_df, edges_df)
    except NameError:
        # 如果变量不在内存中，则从文件加载
        visualize_retrieval()
else:
    logging.info("处于预演模式，跳过可视化。")